In [2]:
#1.1 Initial datapoints. log loss 0.64
#1.2 Momentum factors, CPI/PPI Calculations, modified timeframe. LL: 1yr - 10.64 3M - 1.64

In [25]:
#Imports and API Key

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [26]:
#pulling data from Quandl

data = pd.read_csv('data_for_pull.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [27]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [28]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [29]:
#converting GDP quarterly data into monthly

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

In [30]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)

In [31]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [32]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
dataset = dataset.iloc[552:]
dataset = dataset.iloc[:-59]

In [33]:
y = dataset['Recession'] #splitting off Y

In [34]:
y_shift = y.shift(3) #shifting y to forecast 3 months out

In [35]:
y_shift = y_shift.fillna(0)

In [36]:
dataset = dataset.drop(columns = ['GDP','Recession']) #dropping calc column and recession column from dataset, experimenting with taking out fed funds rate

In [37]:
#substituting mean value in for missing values and adding dummy column to indicate where done

for col in dataset.columns:
    dataclean.clean_zeros(col, dataset)

In [38]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-6])

momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
momentum_cols.remove('CPI')

for i in [1,3,12]:
    for col in momentum_cols:
        dataclean.create_momentum(col,dataset,i)

In [39]:
#CPI Calcs

for i in [1,3,12]:
    for col in ['CPI','PPI']:
        dataclean.infl_momentum(col,dataset,i)

In [40]:
X = dataset

In [41]:
### Data Prep Finished Here ###

In [42]:
X_train = X.iloc[12:550]
X_test = X.iloc[550:]
y_train = y_shift.iloc[12:550]
y_test = y_shift.iloc[550:]

In [43]:
X_train.columns

Index(['PMI', 'UNR', 'YUNR', 'CONS_SENT', 'HOME_SALES', 'PART_TIME',
       'CPPR_PRICE', 'HOUS_PERMS', 'HOUS_STARTS', 'CAP_UTIL', 'PERS_SAVINGS',
       'EXPORTS', 'IMPORTS', 'TRADE_BALANCE', 'INT_RATE', 'US_M2',
       'US_NHOME_SALES', 'PPI', 'CPI', '3YRT', '10 Year Treasury Yield',
       '3 Month Treasury Yield (Bond Equivalent Basis)', 'Spread',
       'HOME_SALES_PXY', 'CPPR_PRICE_PXY', 'HOUS_PERMS_PXY', 'CAP_UTIL_PXY',
       'US_NHOME_SALES_PXY', '3YRT_PXY', 'PMI_1m_shift', 'UNR_1m_shift',
       'YUNR_1m_shift', 'CONS_SENT_1m_shift', 'HOME_SALES_1m_shift',
       'PART_TIME_1m_shift', 'CPPR_PRICE_1m_shift', 'HOUS_PERMS_1m_shift',
       'HOUS_STARTS_1m_shift', 'CAP_UTIL_1m_shift', 'PERS_SAVINGS_1m_shift',
       'EXPORTS_1m_shift', 'IMPORTS_1m_shift', 'TRADE_BALANCE_1m_shift',
       'INT_RATE_1m_shift', 'US_M2_1m_shift', 'US_NHOME_SALES_1m_shift',
       '3YRT_1m_shift', '10 Year Treasury Yield_1m_shift',
       '3 Month Treasury Yield (Bond Equivalent Basis)_1m_shift',
    

In [44]:
model = LogisticRegression()
model.fit(X_train, y_train) #fitting model

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [45]:
probs = model.predict_proba(X_test)

In [46]:
log_loss(y_test, probs)

1.6581475734545759

In [61]:
roc_auc_score(y_test.values, probs[:,1:])

0.7599557522123894

In [48]:
results = pd.DataFrame(probs)
results['actual'] = y_test.values
results.index = y_test.index
pd.set_option('display.float_format', lambda x: '%.3f' % x)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(results)

            0     1  actual
2004-11 1.000 0.000   0.000
2004-12 1.000 0.000   0.000
2005-01 1.000 0.000   0.000
2005-02 1.000 0.000   0.000
2005-03 1.000 0.000   0.000
2005-04 1.000 0.000   0.000
2005-05 1.000 0.000   0.000
2005-06 1.000 0.000   0.000
2005-07 1.000 0.000   0.000
2005-08 1.000 0.000   0.000
2005-09 1.000 0.000   0.000
2005-10 1.000 0.000   0.000
2005-11 1.000 0.000   0.000
2005-12 1.000 0.000   0.000
2006-01 1.000 0.000   0.000
2006-02 1.000 0.000   0.000
2006-03 1.000 0.000   0.000
2006-04 1.000 0.000   0.000
2006-05 1.000 0.000   0.000
2006-06 1.000 0.000   0.000
2006-07 1.000 0.000   0.000
2006-08 1.000 0.000   0.000
2006-09 1.000 0.000   0.000
2006-10 1.000 0.000   0.000
2006-11 1.000 0.000   0.000
2006-12 1.000 0.000   0.000
2007-01 1.000 0.000   0.000
2007-02 1.000 0.000   0.000
2007-03 1.000 0.000   0.000
2007-04 1.000 0.000   0.000
2007-05 1.000 0.000   0.000
2007-06 1.000 0.000   0.000
2007-07 1.000 0.000   0.000
2007-08 1.000 0.000   0.000
2007-09 1.000 0.000 